This script takes a geopackage file, for example from the Dutch CBS, and turns it into a shapefile that can be opened in your preferred GIS software (I prefer QGIS).

The dataset is described by the CBS here: https://www.cbs.nl/nl-nl/longread/diversen/2022/statistische-gegevens-per-vierkant-2021-2020-2019?onepage=true
500x500m datasets can be found here: https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische-data/kaart-van-500-meter-bij-500-meter-met-statistieken
100x100m datasets can be found here: https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische-data/kaart-van-100-meter-bij-100-meter-met-statistieken

For the purpose of this, which is to let high-school students design a new route towards Schiphol, 500m is enough resolution

The geopackage file that can be extracted from the downloaded zip needs to be placed into the input folder that is in the root folder of this file. When this script is run a shapefile (.shp) is placed in the output folder as a result.

In [24]:
# This cell imports needed packages
# Fiona is needed to read the geopackage
# pyproj is needed to shiftfrom the used Dutch Rijksdriehoek to the more widely accepted WGS84
# shapely is used to convert a list to the geometry type that is needed
# pandas and geopandas are used to create the dataframe and export it as a shapefile

import fiona
import pyproj
import shapely
import pandas as pd
import geopandas

Here the input file needs to be defined:

In [22]:
inputfile = "cbs_vk500_2019_vol.gpkg"

Here the data that is exported and used later in the process is selected. For this, the population of a cell is selected, but anything in the dataset can be used.

In [20]:
selected_data = 'aantal_inwoners'

In [25]:
gpkg = "A - Input//" + inputfile
c = fiona.open(gpkg)

# transformer that takes RD coordinates and transforms them to WGS84
forward_transformer = pyproj.Transformer.from_crs(28992, 4326)

#lists for data and coordinates are created
list1 = []
list2 = []

#features are looped through
for feature in c:

    #needed data set is selected
    inwoners = feature['properties'][selected_data]

    #some cells contain negative data, these are set to 0
    if inwoners < 0:
        inwoners = 0
    
    #population is appended to list
    list1.append(inwoners)

    # rectangle needs to be created as a polygon, in order to do this the first coordinate is used twice as it needs to close; hence 5 values
    # these 5 values are extracted and converted from RD coords
    coords1 = forward_transformer.transform(feature['geometry']['coordinates'][0][0][0][0],feature['geometry']['coordinates'][0][0][0][1])
    coords2 = forward_transformer.transform(feature['geometry']['coordinates'][0][0][1][0],feature['geometry']['coordinates'][0][0][1][1])
    coords3 = forward_transformer.transform(feature['geometry']['coordinates'][0][0][2][0],feature['geometry']['coordinates'][0][0][2][1])
    coords4 = forward_transformer.transform(feature['geometry']['coordinates'][0][0][3][0],feature['geometry']['coordinates'][0][0][3][1])
    coords5 = forward_transformer.transform(feature['geometry']['coordinates'][0][0][4][0],feature['geometry']['coordinates'][0][0][4][1])

    #coords are swapped around
    coords1 = coords1[1], coords1[0]
    coords2 = coords2[1], coords2[0]
    coords3 = coords3[1], coords3[0]
    coords4 = coords4[1], coords4[0]
    coords5 = coords5[1], coords5[0]

    #coords are appended to list
    list2.append([coords1,coords2,coords3,coords4,coords5])
        

In [11]:
# list with coordinates is looped through
# per instance each list with 5 entries is turned into a polygon
for i in range(len(list2)):
    list2[i] = shapely.Polygon(list2[i])

In [14]:
# the lists are combined into a dataframe, which is transposed to be correct
df = pd.DataFrame((list1, list2))
df = df.T
# labels are added
df.columns = ["inwoners","Polygon" ]
# dataframe is turned into a geodataframe that can work with geometries
df = geopandas.GeoDataFrame(df, geometry='Polygon', crs='EPSG:4326')
# population is turned into an integer
df.inwoners = pd.to_numeric(df.inwoners)

In [19]:
# geodataframe containing data and coordinates is exported
df.to_file('B - Output//datamap.shp', driver='ESRI Shapefile')